In [13]:
import yfinance as yf
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots


data = yf.download("MSFT",
                   period="1y",
                   interval="1d",
                   progress=False,
                   ignore_tz =True,
                   auto_adjust = True,
                   repair=True)


data = data.reindex(pd.date_range(start=data.index.min(), end=data.index.max(), freq='D', normalize=True))
data.fillna(method='ffill', inplace=True)


#del data["index"]
print(data)

                  Open        High         Low       Close      Volume
2022-02-07  303.327762  304.982242  297.115949  298.156219  28533300.0
2022-02-08  298.453426  302.723413  297.165507  301.732697  32421200.0
2022-02-09  306.993397  309.034271  304.536438  308.320953  31284700.0
2022-02-10  301.217592  306.250421  297.908601  299.572998  45386200.0
2022-02-11  300.375422  301.465217  291.488691  292.301086  39175600.0
...                ...         ...         ...         ...         ...
2023-02-02  258.820007  264.690002  257.250000  264.600006  39940400.0
2023-02-03  259.540009  264.200012  257.100006  258.350006  29062300.0
2023-02-04  259.540009  264.200012  257.100006  258.350006  29062300.0
2023-02-05  259.540009  264.200012  257.100006  258.350006  29062300.0
2023-02-06  259.541992  257.750000  255.690002  256.100006   1965512.0

[365 rows x 5 columns]


In [24]:
data["Median Price"] = (data["Low"] + data["High"])/2
data["5 Median SMA"] = data["Median Price"].rolling(5).mean()
data["34 Median SMA"] = data["Median Price"].rolling(34).mean()
data["Awesome Osc."] = data["5 Median SMA"] - data["34 Median SMA"]
data["Awesome Diff"] = data["Awesome Osc."] - data["Awesome Osc."].shift()

data["Awesome Color"] = data["Awesome Diff"].apply(lambda x: "green" if x>0 else "red" if x<0 else "gray")
data=data.dropna()
print(data)

                  Open        High         Low       Close      Volume  \
2022-04-16  286.005794  286.224204  277.299252  277.805542  28221600.0   
2022-04-17  286.005794  286.224204  277.299252  277.805542  28221600.0   
2022-04-18  276.892202  280.416507  276.326318  278.490540  20778000.0   
2022-04-19  277.358808  284.099693  276.395824  283.235962  22297700.0   
2022-04-20  287.306284  287.604132  283.305441  284.288269  22906700.0   
...                ...         ...         ...         ...         ...   
2023-02-02  258.820007  264.690002  257.250000  264.600006  39940400.0   
2023-02-03  259.540009  264.200012  257.100006  258.350006  29062300.0   
2023-02-04  259.540009  264.200012  257.100006  258.350006  29062300.0   
2023-02-05  259.540009  264.200012  257.100006  258.350006  29062300.0   
2023-02-06  259.541992  257.750000  255.690002  256.100006   1965512.0   

            Median Price  5 Median SMA  34 Median SMA  Awesome Osc.  \
2022-04-16    281.761728    282.342480  

In [28]:
fig2 = make_subplots(rows=2, cols=1, row_heights = [0.7, 0.3])

fig2.add_trace(go.Scatter(x=data.index, y=data["Close"], showlegend=False), col=1, row=1)
fig2.add_trace(go.Bar(x=data.index, y=data["Awesome Osc."], marker_color = data["Awesome Color"], showlegend=False), col=1, row=2)

fig2.update_layout(height=1000, width=1000)
fig2.show()